In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

#### SF 2018

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220228/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz025 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.25-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.5-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz150 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.5-20220328/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/"

#### SF 2019

In [3]:
loc_2019_transCap05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_capacity_0.5-20220329/beam/year-2019-iteration-5/ITERS/it.0/"

In [4]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_baseline + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:20: DtypeWarning: Columns (5,7,12,13,14,15,17,23,29,32,44,53,55,56) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 46s


In [5]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
eventsSF['scenario'] = "baseline"
eventsSF['year'] = '2018'

In [7]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [8]:
eventsSFCopy = eventsSF.copy()

In [9]:
# deadhead = eventsSF[eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)
# When there are no riders. Driver and person are not numbers, but include "Agent"

In [9]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [10]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [11]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [12]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [13]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [14]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [15]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

Wall time: 1min 48s


In [16]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [17]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [18]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [19]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3380, 65)

In [20]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

Wall time: 4min 29s


In [21]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [22]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(0, 66)

In [23]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [24]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', nan, 'car', 'car_hov2', 'walk', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bike_transit']


In [25]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', nan, 'car', 'walk', 'car_hov2', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [26]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'TeleportationEvent', 'arrival', 'actstart', 'PersonEntersVehicle', 'PathTraversal', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'PersonLeavesVehicle', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [27]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

Wall time: 18.5 s


In [28]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [29]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [30]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [31]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [32]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [33]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [34]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [35]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [36]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [37]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding census blocks

In [73]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [74]:
BGs = gpd.read_file('C:/Shared-Work/Data/Scenario_2010_shp/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [75]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'startBlockGroup')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Wall time: 1h 2min 23s


In [76]:
eventsSF.head()

,IDMerged,person,driver,riders,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,price,endY,endX,arrivalTime,departureTime,primaryFuelLevel,secondaryFuelLevel,cost,links,numPassengers,primaryFuel,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,departTime,score,incentive,tollCost,cost_BEAM,scenario,year,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status,transit_bus,transit_subway,transit_tram,transit_rail,transit_cable_car,ride_hail_pooled,startBlockGroup
0,1,1,NaN,NaN,NaN,29037.0,ModeChoice,hov3_teleportation,hov3_teleportation,NaN,HOV3_TELEPORTATION,56792.0,true,6498.758,1.0,"WALK,CAR_HOV3,WALK","body-1,teleportationSharedVehicle-1-1,body-1",Home,othmaint,577.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,hov3_teleportation,NaN,NaN,NaN,6498.758,6498.758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6498.758,NaN,NaN,NaN,1.0,,,577.0,hov3_teleportation,hov3_teleportation,0,0,0,0,0,0,0,NaN
1,1,1,NaN,NaN,NaN,29037.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56792.0,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,29037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,577.0,NaN,NaN,0,0,0,0,0,0,0,NaN
2,1,1,NaN,NaN,NaN,29037.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56792.0,hov3_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,577.0,NaN,NaN,0,0,0,0,0,0,0,NaN
3,1,1,NaN,NaN,NaN,29601.0,TeleportationEvent,NaN,hov3_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.377254,-121.802441,29601.0,29037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,hov3_teleportation,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,,,577.0,NaN,NaN,0,0,0,0,0,0,0,060855035091
4,1,1,NaN,NaN,NaN,29601.0,arrival,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65790.0,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,577.0,NaN,NaN,0,0,0,0,0,0,0,NaN


In [ ]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'endBlockGroup')

#### Adding new columns

In [38]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [39]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [40]:
eventsSF['travelTime'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [41]:
eventsSF['travelDistance'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [42]:
eventsSF['length_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [43]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelTime'], np.nan)

In [44]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelDistance'], np.nan)

In [45]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelTime'], np.nan)

In [46]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelDistance'], np.nan)

In [47]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['travelTime'], np.nan)

In [48]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [49]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['travelTime'], np.nan)

In [50]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['travelDistance'], np.nan)

In [51]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelTime'], np.nan)

In [52]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [53]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [84]:
#eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                #eventsSF['primaryFuel'], np.nan)

In [85]:
#eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                #eventsSF['primaryFuel'], np.nan)

In [57]:
#eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                #eventsSF['primaryFuel'], np.nan)

In [58]:
#eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                #eventsSF['primaryFuel'], np.nan)

In [59]:
#eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            #, eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [60]:
#eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           #eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [54]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [55]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [56]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Previous code for mode choice 

In [64]:
# Make a new column which determined mode choice numbers for each person and trip
#s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [65]:
#eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [66]:
#eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [67]:
# Just showing the mode choice after replanning
#eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

#### Only works if we have one replanning event

In [68]:
# Showing both mode choices before and after the replanning event
#eventsSF['mode_choice_act'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [69]:
# If replanned, then the status is 1
#eventsSF['replanning_status'] = np.where(eventsSF['mode_choice_replanned_BEAM'].notna() , 1, np.nan)

In [70]:
# Group by and fill all the rows related to the replanning event
#eventsSF["replanning_status"] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False)['replanning_status'].transform(lambda x: x.ffill().bfill())

In [71]:
# Put all non-replanning events equal to 0
#eventsSF["replanning_status"] = np.where(eventsSF['replanning_status'].isna(), 0, eventsSF['replanning_status']).astype(int)

In [72]:
#eventsSF['mode_choice_actual_BEAM'] = np.where((eventsSF['replanning_status'] == 1) & (eventsSF['type_number']=='ModeChoice01'), np.nan, eventsSF['mode_choice_act'])

In [73]:
#eventsSF = eventsSF.drop(['mode_choice_act'], axis = 1)

In [74]:
#eventsSF[eventsSF['type'] == "Replanning"].tail(16)

In [76]:
#num_pass.to_csv('C:/Shared-Work/Data/CleanData/num_pass.csv', index = False)

In [77]:
#eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

In [78]:
#eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [79]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [80]:
#eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [81]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

#### Mode Choice planned and actual

In [57]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

Wall time: 1min 26s


In [58]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

Wall time: 1min 24s


In [59]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [60]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [61]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [62]:
#eventsSF[eventsSF['type']=='Replanning'].head()

In [63]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [64]:
#q = eventsSF[eventsSF['IDMerged'] == 6902751]

In [65]:
#q.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/fake2.csv')   

In [66]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

Wall time: 1.7 s


In [67]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [68]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [103]:
#transit_data = eventsSF[(eventsSF['modeBEAM_rh'] == "bus") | (eventsSF['modeBEAM_rh'] == "subway") 
#                        | (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'ride_hail_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'bike_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'cable_car') | (eventsSF['modeBEAM_rh'] == 'rail') 
#                        | (eventsSF['modeBEAM_rh'] == 'tram')]

In [ ]:
#eventsSF.loc['Total',:]= df.sum(axis=0)

In [ ]:
#eventsSF.type[eventsSF.type == 'Replanning'].count()

In [ ]:
#Person_Trip_eventsSF.replanning_status[Person_Trip_eventsSF.replanning_status == 0].count()

In [ ]:
#rh_transit5 = eventsSF[eventsSF['IDMerged']== 3671596]

In [ ]:
#rh_transit5.to_csv('C:/Shared-Work/Data/CleanData/rh_transit5.csv', index = False)

In [69]:
eventsSFCopy = eventsSF.copy()

In [78]:
eventsSF.head(100000).to_csv('C:/Shared-Work/Data/CleanData/modeBEAMCHECK.csv', index = False)

In [163]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'travelTime': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'travelDistance': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'length_mode_choice': np.sum, 
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 'transit_subway': np.sum, 'transit_tram': np.sum, 'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum,
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

Wall time: 28min 7s


In [164]:
Person_Trip_eventsSF['door_to_door_time'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [165]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['door_to_door_time'] - Person_Trip_eventsSF['travelTime'] 

In [166]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [113]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged'] == 2192]

In [ ]:
#Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [167]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"legVehicleIds_estimate"}, inplace=True) 

In [168]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [116]:
#negative_door.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/negative_door.csv', index = False)   

In [ ]:
#Person_Trip_eventsSF.door_to_door_time[Person_Trip_eventsSF.door_to_door_time<0].count()

In [ ]:
#negative_door = Person_Trip_eventsSF[(Person_Trip_eventsSF.door_to_door_time<0)]

In [ ]:
#a  = Person_Trip_eventsSF.describe()

In [ ]:
#negative_door.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/negative_door.csv')   

In [ ]:
#sample_bike2 = eventsSF[(eventsSF['primaryFuelType'] == "Gasoline") & (eventsSF['modeBEAM'] == "bike") ]

In [169]:
Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base.csv', index = False)

In [ ]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['mode_choice_planned_BEAM'] == 'walk_transit'].head()

In [170]:
Person_Trip_eventsSF.head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_privateCar,distance_ridehail,distance_transit,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_walking,legVehicleIds_estimate,length_mode_choice,mode_choice_actual_BEAM,mode_choice_planned_BEAM,numPassengers,reason,replanning_status,ride_hail_pooled,scenario,transit_bus,transit_cable_car,transit_rail,transit_subway,transit_tram,travelDistance,travelTime,vehicleIds,year,door_to_door_time,waitTime,actPurpose
0,1,577.0,29037.0,Home,29601.0,othmaint,0.000000,0.0,6498.758,0.0,0.0,0.000,564.0,0.0,0.0,0.0,0.0,"body-1,teleportationSharedVehicle-1-1,body-1",6498.758,hov3_teleportation,hov3_teleportation,,,0,0,baseline,0,0,0,0,0,6498.758,564.0,,2018,564.0,0.0,Home_to_othmaint
1,1,578.0,29751.0,othmaint,29890.0,school,0.000000,0.0,1641.981,0.0,0.0,0.000,139.0,0.0,0.0,0.0,0.0,"body-1,teleportationSharedVehicle-2-1,body-1",1641.981,hov3_teleportation,hov3_teleportation,,,0,0,baseline,0,0,0,0,0,1641.981,139.0,,2018,139.0,0.0,othmaint_to_school
2,1,581.0,52812.0,school,53394.0,Home,0.000000,0.0,6326.206,0.0,0.0,0.000,582.0,0.0,0.0,0.0,0.0,"body-1,teleportationSharedVehicle-3-1,body-1",6326.206,hov3_teleportation,hov3_teleportation,,,0,0,baseline,0,0,0,0,0,6326.206,582.0,,2018,582.0,0.0,school_to_Home
3,2,689.0,46533.0,work,48475.0,atwork,3.656274,0.0,33336.011,0.0,0.0,786.423,1337.0,0.0,0.0,0.0,605.0,"body-2,539168,539168,body-2",34122.434,car,car,"1.0, 0.0, 0.0, 1.0",,0,0,baseline,0,0,0,0,0,34122.434,1942.0,"539168, body-2",2018,1942.0,0.0,work_to_atwork
4,2,693.0,49010.0,atwork,49126.0,work,0.182227,0.0,1661.452,0.0,0.0,30.277,93.0,0.0,0.0,0.0,23.0,"body-2,539168,539168,body-2",1691.729,car,car,"1.0, 0.0, 0.0, 1.0",,0,0,baseline,0,0,0,0,0,1691.729,116.0,"539168, body-2",2018,116.0,0.0,atwork_to_work


#### Merging with activity sim persons and housholds files

In [172]:
actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/activitysim/"
actloc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220228/activitysim/"
actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/activitysim/"
actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

households = pd.read_csv(actloc_2018_baseline + 'final_households.csv')
persons = pd.read_csv(actloc_2018_baseline + 'final_persons.csv')
tours = pd.read_csv(actloc_2018_baseline +'final_tours.csv')
plans = pd.read_csv(actloc_2018_baseline +'final_plans.csv')
trips = pd.read_csv(actloc_2018_baseline + 'final_trips.csv')

In [173]:
#persons.earning[persons.earning<0].count()

In [174]:
# Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [175]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [176]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [177]:
trips.head()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
507811,577,1,1735309,72,school,1,True,2,othmaint,595,641,5.730384,8.0,SHARED2FREE,-2.150332
507812,578,1,1735309,72,school,2,True,2,school,633,595,NaN,8.0,SHARED2FREE,-2.020166
507813,581,1,1735309,72,school,1,False,1,Home,641,633,NaN,14.0,SHARED2FREE,-1.999284
507814,689,2,1735309,86,atwork,1,True,1,atwork,700,700,NaN,12.0,DRIVEALONEFREE,-0.505729
507815,693,2,1735309,86,atwork,1,False,1,Work,700,700,NaN,14.0,DRIVEALONEPAY,-0.507800


In [180]:
# Merge person_trip level BEAM with activity sim merged files
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id', 'trip_id'])
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex'])
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [181]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [182]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [183]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [184]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [94]:
# eventsASim.head(10000).to_csv('C:/Shared-Work/Data/CleanData/Merged_SF_Baseline_2018.csv', index = False)

In [185]:
eventsASim.to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/sf_2018_base.csv', index = False)

In [ ]:
# eventsASim.duration_in_privateCar.sum()

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

#plt.figure(figsize=(13, 6))
#sns.heatmap(households, vmax=1, annot=True, linewidths=.5)
#plt.xticks(rotation=30, horizontalalignment='right')
#plt.show()

In [ ]:
#import missingno as msno
#%matplotlib inline
#msno.bar(SFbaseline2018)
#msno.heatmap(persons)

In [ ]:
#from io import StringIO # python3; python2: BytesIO 
#import boto
#import boto3
#
#conn = boto.s3.connect_to_region('us-east-2')
#bucket = conn.get_bucket('beam-core-act')
#csv_buffer = StringIO()
#eventsASim.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
#s3_resource.Object(bucket, 'eventsASim.csv').put(Body=csv_buffer.getvalue())